# MOM6/CESM Ocean Model Analysis

This notebook shows how to load and analyze ocean data from an out-of-the-box MOM6/CESM G-case simulation (coupled ocean ocean/sea ice). 

**NOTE**: MOM6/CESM is not ready to be used for research. 

In [ ]:
%matplotlib inline

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import holoviews as hv
import datashader
from holoviews.operation.datashader import regrid, shade, datashade

hv.extension('bokeh', width=100)

## Create and Connect to Dask Distributed Cluster

This will launch a cluster of virtual machines in the cloud.

In [ ]:
from dask.distributed import Client, progress
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=10)
cluster

👆 Don't forget to click this link to get the cluster dashboard

In [ ]:
client = Client(cluster)
client

## Load MOM6/CESM Data

This data is stored in [xarray-zarr](http://xarray.pydata.org/en/latest/io.html#zarr) format in Google Cloud Storage.
This format is optimized for parallel distributed reads from within the cloud environment.

It may take up to a minute to initialize the dataset when you run this cell.

In [ ]:
# Load with Cloud object storage
import gcsfs
gcsmap = gcsfs.mapping.GCSMap('pangeo-data/MOM6.CESM/')
ds = xr.open_zarr(gcsmap, decode_cf=True, decode_times=False)

# Print dataset
ds

## Visualize SST Data with Holoviews and Datashader

The cells below show how to interactively explore the dataset.

In [ ]:
sst_ds = hv.Dataset(ds['SST'])
sst = sst_ds.to(hv.QuadMesh, kdims=["xh", "yh"], dynamic=True)
%opts RGB [width=1000 height=600] 
datashade(sst, precompute=True, cmap=plt.cm.jet)

## Visualize SSS Data with Holoviews and Datashader

In [ ]:
sss_ds = hv.Dataset(ds['SST'])
sss = sss_ds.to(hv.QuadMesh, kdims=["xh", "yh"], dynamic=True)
datashade(sss, precompute=True, cmap=plt.cm.Spectral_r)

## Data reduction

Here we make a data reduction by taking the time of SST and SSS. This demonstrates how the cluster distributes the reads from storage.

In [ ]:
SST_mean = ds.SST.mean(dim=('Time'))
SST_mean

In [ ]:
SSS_mean = ds.SSS.mean(dim=('Time'))
SSS_mean

In [ ]:
%time SST_mean.load()

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
SST_mean.plot.contourf(levels=np.arange(-2,30))
plt.title('Mean SST')

In [ ]:
%time SSS_mean.load()

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
SSS_mean.plot.contourf(levels=np.linspace(31,38,20),cmap=plt.cm.jet)
plt.title('Mean SSS')